In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [4]:
base = pd.read_csv("stage3.csv")

In [136]:
pd.set_option('display.max_columns', None)
base

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,0::Julian Sims,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,0::Bernard Gillis,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,0::Unknown||1::Unknown,0::Unknown||1::Unknown,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,0::Damien Bell||1::Desmen Noble||2::Herman Sea...,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,0::Stacie Philbrook||1::Christopher Ratliffe||...,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,0::Unknown||1::Unknown,0::Handgun||1::Handgun,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,0::Danielle Imani Jameison||1::Maurice Eugene ...,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,1083142,2018-03-31,Louisiana,Rayne,North Riceland Road and Highway 90,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.klfy.com/news/local/rayne-woman-cha...,False,NaN,0::Unknown,0::Unknown,Shots Fired - No Injuries,NaN,NaN,NaN,1.0,NaN,0::25,0::Adult 18+,0::Female,0::Jhkeya Tezeno,NaN,"0::Unharmed, Arrested",0::Subject-Susp

In [6]:
def convert_to_dict(value):
    if pd.isna(value):
        return value

    if '||' not in str(value):
        result_dict={0:value}
        return result_dict

    if '||' in str(value):
        pairs = value.split('||')
        pairs[1].strip("'])")

        result_dict = {}
        for pair in pairs:
            #Some are corrupted : 1: instead of ::
            if '::' in pair:
                key, val = pair.split('::', 1)
                result_dict[int(key)] = val
            else:
                key, val = pair.split(':', 1)
                result_dict[int(key)] = val
        return result_dict


updated_base = base.copy()
list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict)
updated_base.head()



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


Beaucoup des dictionnaires ne se sont pas bien convertis en raison de corruption de la base : certains sont codés avec '|' à la place de '||', nous essayons donc ici de gérer ces exceptions.

In [7]:

def convert_to_dict2(value2):
    if pd.isna(value2):
        return value2

    #some dict are corrupted : one | instead of ||
    list_value=list(value2.values())
    str_value= ''.join(list_value)

    if '|' not in str_value:
        return value2

    if '|' in str_value:
        
        pairs = str_value.split('|')
        pairs[1].strip("'])")

        result_dict = {}
        for pair in pairs:
            #Some are corrupted : 1: instead of ::
            if '::' in pair:
                key, val = pair.split('::', 1)
                result_dict[int(key)] = val
            elif ':' in pair:
                key, val = pair.split(':', 1)
                result_dict[int(key)] = val
            #because they already passed through convert_to_dict, the first key (0) is missing
            else:
                key, val=0,pair
                result_dict[key]=val
                print(val)
        return result_dict

list_of_dict_columns = ['gun_stolen', 'gun_type', 'participant_age', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type']
updated_base[list_of_dict_columns] = updated_base[list_of_dict_columns].applymap(convert_to_dict2)
updated_base.head()


,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://pittsburgh.cbslocal.com/2013/01/01/4-pe...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",http://losangeles.cbslocal.com/2013/01/01/man-...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",http://www.morningjournal.com/general-news/201...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://denver.cbslocal.com/2013/01/06/officer-...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",http://myfox8.com/2013/01/08/update-mother-sho...,62.0,27.0


In [8]:
def convert_to_list(value):
    if pd.isna(value):
        return value

    if '||' in str(value):
        liste = value.split('||')
        return liste


list_of_list_columns = ['incident_characteristics', 'sources']
updated_base[list_of_list_columns] = updated_base[list_of_list_columns].applymap(convert_to_list)
updated_base.head()



,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district
0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN
1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0
2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0
3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0
4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0


### Début des statistiques descriptives 

import des packages nécessaires:

In [9]:
from plotly.offline import init_notebook_mode
init_notebook_mode(connected= True)

In [10]:
import plotly.express as px

### Barplot décrivant le nombre d'incidents par arme à feu par Etat

In [11]:
fig = px.bar(updated_base, x=sorted(pd.unique(updated_base['state'])), y=updated_base.groupby('state').count()['incident_id'], title="Nombre d'incidents par arme à feu par Etat",
labels={"x":"Etat", "y":"nombre d'incidents"})

In [12]:
fig.show()

On remarque ici que les Etats dans lesquels le plus d'incidents par arme à feu ont lieu sont : l'Illinois (17.5k), la Californie (16.3k), la Floride (15k) et le Texas (13.5k). 
## A vérifier si cohérent avec la littérature sur le sujet 

### Graphique étudiant le nombre d'incidents par mois 

In [13]:
updated_base['date']=pd.to_datetime(updated_base['date'])

In [14]:
updated_base['month_year'] =updated_base['date'].dt.month.astype(str)+"-"+updated_base['date'].dt.year.astype(str)

In [15]:

fig2 = px.bar(updated_base, x=pd.unique(updated_base['month_year']), y=updated_base.groupby(updated_base['month_year']).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu par mois")


fig2

In [16]:

fig2bis = px.bar(updated_base, x=pd.unique(updated_base['date'].dt.year), y=updated_base.groupby(updated_base['date'].dt.year).count()['incident_id'],
              labels={"x":"date", "y":"nombre d'incidents"},
              title="Nombre d'incidents par arme à feu par an")


fig2bis

On remarque que tous les 5 ou 6 mois très peu d'incidents sont renseignés : problème de récolte des données ? saisonnalité peu probable au vu de la variation des mois concernés.
De plus la base de données est incomplète en 2013 et en 2018, mais on observe tout de même une augmentation du nombre d'incidents sur les 4 années complètes de la base. 

### étude du nombre de morts et blessés par incident

In [17]:
fig3 = px.bar(updated_base, x=sorted(pd.unique(updated_base['n_killed'])), y=updated_base.groupby(updated_base['n_killed']).count()['incident_id'],
              labels={"x":"nombre de morts", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre d'incidents ayant entraîné des morts",
              log_y=True)

fig3

On voit ici que la grande majorité des incidents n'entraînent pas de mort (185.5k) et que près de 50.000 incidents provoquent une mort. Quelques outliers provoquent au delà de 10 morts, ils correspondent aux tueries dites "de masse" dont voici les références ci-dessous.

In [18]:
mass_kill=updated_base[updated_base['n_killed']>=10]
mass_kill

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year
194,480366,2013-09-16,District of Columbia,Washington Navy Yard,1336 Isaac Hull Ave,11,3,http://www.gunviolencearchive.org/incident/480366,https://www.washingtonpost.com/local/police-se...,False,1.0,"{0: 'Unknown', 1: 'Unknown'}","{0: '9mm', 1: 'Shotgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",38.8730,NAVSEA Building 197,-76.9977,2.0,"Male victim shot in the leg, first female vict...","{0: '59', 1: '53', 2: '51', 3: '53', 4: '62', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Female',...","{0: 'Michael Arnold', 1: 'Martin Bodrog', 2: '...",{0: '15::Co-worker'},"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.nbcwashington.com/news/local/Confi...,NaN,NaN,9-2013
92624,423223,2015-10-01,Oregon,Roseburg,1140 Umpqua College Rd,10,9,http://www.gunviolencearchive.org/incident/423223,http://www.kval.com/news/local/Nine-wounded-vi...,False,4.0,"{0: 'Unknown', 1: 'Unknown', 2: 'Unknown', 3: ...","{0: '223 Rem [AR-15]', 1: 'Handgun', 2: 'Handg...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",43.2628,Umpqua Community College,-123.2800,6.0,shots fired at Snyder Hall around 10:30 a.m. 7...,"{0: '19', 1: '18', 2: '59', 3: '18', 4: '33', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Female', 3: 'Male...","{0: 'Lucero Alcaraz', 1: 'Quinn Glen Cooper', ...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.statesmanjournal.com/story/news/20...,7.0,4.0,10-2015
101531,456893,2015-12-02,California,San Bernardino,1365 South Waterman Avenue,16,19,http://www.gunviolencearchive.org/incident/456893,http://www.latimes.com/local/lanow/la-me-ln-sa...,False,31.0,"{0: 'Not-stolen', 1: 'Not-stolen', 2: 'Not-sto...","{0: '223 Rem [AR-15]', 1: '223 Rem [AR-15]', 2...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",34.0758,Inland Regional Center,-117.2770,4.0,Second Location [police v shooter] 1700 block ...,"{0: '42', 1: '58', 2: '52', 3: '37', 4: '40', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Male', 5...","{0: 'Larry Daniel Kaufman', 1: 'Damian Meins',...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.sbsun.com/general-news/20151202/14...,40.0,23.0,12-2015
130448,577157,2016-06-12,Florida,Orlando,1912 S Orange Avenue,50,53,http://www.gunviolencearchive.org/incident/577157,https://www.reuters.com/article/us-orlando-sho...,False,5.0,"{0: 'Not-stolen', 1: 'Not-stolen'}","{0: '223 Rem [AR-15]', 1: '9mm'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",28.5195,Pulse,-81.3767,2.0,Names are coming in sporadically and every eff...,"{0: '34', 1: '23', 2: '20', 3: '22', 4: '36', ...","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Edward Sotomayor Jr', 1: 'Stanley Almodov...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.cityoforlando.net/police/mass-shoo...,47.0,12.0,6-2016
217151,980577,2017-11-05,Texas,Sutherland Springs,216 4th St,27,20,http://www.gunviolencearchive.org/incident/980577,http://abc7.com/full-list-of-texas-church-shoo...,False,28.0,"{0: 'Unknown', 1: 'Unknown', 2: 'Unknown', 3: ...","{0: '223 Rem [AR-15]', 1: 'Rifle', 2: '9mm', 3...","[Shot - Wounded/Injured, Shot - Dead (murder, ...",29.2733,First Ba

In [19]:
fig4 = px.bar(updated_base, x=sorted(pd.unique(updated_base['n_injured'])), y=updated_base.groupby(updated_base['n_injured']).count()['incident_id'],
              labels={"x":"nombre de blessés", "y":"nombre d'incidents (échelle logarithmique)"},
              title="Nombre de blessés par incident",
              log_y=True)

fig4

Le nombre de blessés par incidents suit globalement la même distribution que le nombre de morts par incident mais est plus étalée. Dans 142.5k cas il n'y a aucun blessé, mais dans 81.000 incidents il y a un blessé. Les incidents provoquant plus de 10 blessés sont rares, les voici dans un dataframe. On y retrouve les incidents très meurtriers (Orlando, Sutherland Springs) mais également d'autres n'ayant pas fait beaucoup de morts.

In [20]:
mass_injured=updated_base[updated_base['n_injured']>10]
mass_injured

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year
34,482771,2013-03-11,District of Columbia,Washington,1200 North Capitol Street,0,13,http://www.gunviolencearchive.org/incident/482771,https://www.washingtonpost.com/local/police-se...,False,1.0,"{0: 'Unknown', 1: 'Unknown', 2: 'Unknown', 3: ...","{0: '9mm', 1: '9mm', 2: '40 SW', 3: '40 SW'}","[Shot - Wounded/Injured, Drive-by (car to stre...",38.9062,Tyler House,-77.0099,4.0,Two .40 caliber weapons used may have been the...,"{0: '19', 1: '19', 2: '17', 15: '21'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Craig Steven Wilson', 1: 'Andrew Davon Al...",NaN,"{0: 'Unharmed, Arrested', 1: 'Unharmed, Arrest...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.washingtontimes.com/news/2013/may/...,NaN,NaN,3-2013
73,486209,2013-05-12,Louisiana,New Orleans,Frenchmen Street,0,19,http://www.gunviolencearchive.org/incident/486209,http://www.nola.com/crime/index.ssf/2015/09/mo...,False,2.0,{0: '0::Unknown'},{0: '0::7.62 [AK-47]'},"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",29.9861,NaN,-90.0595,1.0,ms; 19 inj. Rival gang members retaliation eff...,"{17: '10', 18: '10', 19: '19', 20: '23'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'JT', 19: 'Akein Scott', 20: 'Shawn Scott'...",NaN,"{0: 'Injured', 1: 'Injured', 2: 'Injured', 3: ...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.upi.com/Top_News/US/2013/05/12/Abo...,97.0,4.0,5-2013
200,480521,2013-09-19,Illinois,Chicago,1809 W 50th St,0,12,http://www.gunviolencearchive.org/incident/480521,http://www.nydailynews.com/news/national/chica...,False,3.0,{0: '0::Unknown'},{0: '0::7.62 [AK-47]'},"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",41.8030,Cornell Square Park,-87.6699,1.0,"Sources said the ""Brick Squad"" gang faction, a...","{0: '3', 1: '17', 2: '15', 3: '27', 4: '24', 5...","{0: 'Child 0-11', 1: 'Teen 12-17', 2: 'Teen 12...","{0: 'Male', 1: 'Female', 2: 'Male', 3: 'Male',...","{0: 'Deonta Howard', 13: 'Quinton M. Humphries...","{13: 'Gang vs Gang', 14: 'Gang vs Gang', 15: '...","{0: 'Injured', 1: 'Injured', 2: 'Injured', 3: ...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.chicagotribune.com/news/local/brea...,2.0,1.0,9-2013
213,480765,2013-10-05,California,Fresno,3883 E. Calwa #114,1,12,http://www.gunviolencearchive.org/incident/480765,http://sanfrancisco.cbslocal.com/2013/10/06/gu...,False,16.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.7054,Soul Brothers Clubhouse,-119.7600,NaN,NaN,{0: '0::18'},"{0: 'Adult 18+', 2: 'Adult 18+', 3: 'Adult 18+...",{0: '0::Male'},{0: '0::Dejuan Gladney'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",None,31.0,14.0,10-2013
239,493842,2013-11-09,Texas,Cypress,7300 block of Enchanted Creek Drive,2,16,http://www.gunviolencearchive.org/incident/493842,http://www.dailymail.co.uk/news/article-250059...,False,10.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",29.8840,NaN,-95.7187,NaN,Birthday party in home advertised on social me...,"{0: '17', 1: '16', 2: '16', 18: '21', 19: '18'}","{0: 'Teen 12-17', 1: 'Teen 12-17', 2: 'Teen 12...","{0: 'Male', 1: 'Female', 2: 'Male', 18: 'Male'...","{0: 'Queric Richardson', 1: 'Arielle Shepherd'...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Injured', 3: 'I...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://www.cbsnews.com/pictures/victims-of-th...,132.0,18.0,11-2013
6831,108626,2014-02-20,Florid

### Statistiques descriptives sur les suspects et auteurs des incidents 

Définition d'une fonction qui permet de récupérer les labels récurrents présents dans les dictionnaires.

In [61]:
def unique_label(var):
    liste=[]
    for i in updated_base[var]:
        if type(i)!=dict:
            continue
        else:
            liste.append(i.values())

    flat_list = [item for sublist in liste for item in sublist] 
    print(var, pd.unique(flat_list))

var_dict=['participant_status','participant_type','participant_age_group','participant_age',
    'participant_gender','gun_type', 'gun_stolen','participant_relationship']

for name in var_dict:
    unique_label(name)

participant_status ['Arrested' 'Injured' 'Killed' 'Injured, Unharmed, Arrested'
 'Unharmed, Arrested' 'Unharmed' 'Injured, Arrested'
 'Killed, Unharmed, Arrested' 'Injured, Unharmed' 'Killed, Injured'
 '0::Injured' '0::Killed' '0::Unharmed' '0::Unharmed, Arrested'
 '1::Unharmed' '1::Killed' 'Killed, Unharmed' '0::Injured, Arrested'
 'Killed, Arrested' '1::Injured' '1::Unharmed, Arrested' '0:Injured'
 '0:Killed' '0:Unharmed' '1:Unharmed' '0:Unharmed, Arrested'
 '1:Unharmed, Arrested' '1:Killed' '0:Arrested' '1:Injured' '2::Unharmed'
 '0::Arrested' '2::Killed' '4::Unharmed, Arrested' '2::Arrested'
 '3::Unharmed' '4::Unharmed' '0::Killed, Unharmed' '1::Injured, Arrested'
 '2::Unharmed, Arrested' '0::Killed, Arrested' '2::Injured' '1::Arrested'
 '0::Injured, Unharmed' '0::Injured, Unharmed, Arrested'
 '0::Killed, Unharmed, Arrested' '3::Unharmed, Arrested'
 '2::Injured, Arrested' '3::Injured' '1::Killed, Unharmed'
 '0::Killed, Injured']
participant_type ['Victim' 'Subject-Suspect' '0::Subj

Les dictionnaires ne sont pas parfaits au moment où je commence ces statistiques au vu des valeurs uniques mais le nombre d'erreurs a été considérablement réduit et paraît négligeable au vu de la taille de la base de données.
## attention c'est pas tout à fait ça !!

In [62]:
data_suspects=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Subject-Suspect':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_sus']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_sus']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_sus']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['gender_sus']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['gender_sus']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_sus']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_sus']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan
                
                if type(updated_base['participant_relationship'][index])==dict:
                    if key in updated_base['participant_relationship'][index].keys():
                        result_dict['relation']=updated_base['participant_relationship'][index][key]
                else:
                    result_dict['relation']=np.nan


                data_suspects.append(result_dict)

    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Subject-Suspect':
            result_dict={'id':index}

            if type(updated_base['participant_age_group'][index])==dict:
                if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_sus']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_sus']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_sus']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_sus']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['gender_sus']=updated_base['participant_gender'][index][key]
            else:
                result_dict['gender_sus']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_sus']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_sus']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan

            if type(updated_base['participant_relationship'][index])==dict:
                if key in updated_base['participant_relationship'][index].keys():
                    result_dict['relation']=updated_base['participant_relationship'][index][key]
            else:
                result_dict['relation']=np.nan
                
            data_suspects.append(result_dict)

    index+=1


df_suspects=pd.DataFrame(data_suspects)

In [63]:
df_suspects.head()

,id,age_group_sus,gender_sus,status_sus,gun_type,gun_stolen,relation,age_sus
0,0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,Adult 18+,Male,"Injured, Unharmed, Arrested",Unknown,Unknown,NaN,25
3,2,Adult 18+,Male,"Unharmed, Arrested",Unknown,Unknown,NaN,31
4,3,Adult 18+,Male,Killed,NaN,NaN,NaN,33


On obtient ici un dataframe permettant d'obtenir l'âge, le genre, le status et l'arme de tous les participants aux incidents caractérisés comme 'Subject-Suspect' afin de pouvoir faire des statistiques descriptives. La colonne 'id' corresponds à l'index de l'incident concerné dans la base updated_base. La colonne 'relation' correspond à la relation du suspect à la victime, ou la circonstance de l'incident. 


In [24]:
#il reste environ 10 erreurs de label concernant l'âge, on se permet donc d'ignorer le problème en faisant sur une base filtrée n'ayant que les bonnes catégories
groupes=['Adult 18+', 'Teen 12-17','Child 0-11']
filtered_suspects_age=df_suspects[df_suspects['age_group_sus'].isin(groupes)]

fig5 = px.bar(filtered_suspects_age, x=sorted(pd.unique(filtered_suspects_age['age_group_sus'])), y=filtered_suspects_age.groupby('age_group_sus').count()['id'],
              labels={"x":"âge du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par tranche d'âge")

fig5

In [25]:
age=list(range(10,90))
age=[str(num) for num in age]

filtered_suspects_age2=df_suspects[df_suspects['age_sus'].isin(age)]
filtered_suspects_age2

fig8 = px.bar(filtered_suspects_age2, x=sorted(pd.unique(filtered_suspects_age2['age_sus'])), y=filtered_suspects_age2.groupby('age_sus').count()['id'],
            labels={"x":"âge du suspect", "y":"nombre de suspects"},
            title="Nombre de suspect par tranche d'âge")

fig8

In [64]:
gender=['Male', 'Female']
filtered_suspects_genre=df_suspects[df_suspects['gender_sus'].isin(gender)]

fig6 = px.bar(filtered_suspects_genre, x=sorted(pd.unique(filtered_suspects_genre['gender_sus'])), y=filtered_suspects_genre.groupby('gender_sus').count()['id'],
              labels={"x":"genre du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par genre")

fig6

In [27]:
statut=['Injured','Injured, Unharmed, Arrested',
       'Unharmed, Arrested', 'Killed', 'Unharmed', 'Injured, Arrested',
       'Killed, Unharmed, Arrested', 'Arrested', 'Injured, Unharmed','Killed, Unharmed',
       'Killed, Arrested', 'Killed, Injured']
filtered_suspects_status=df_suspects[df_suspects['status_sus'].isin(statut)]

fig7 = px.bar(filtered_suspects_status, x=sorted(pd.unique(filtered_suspects_status['status_sus'])), y=filtered_suspects_status.groupby('status_sus').count()['id'],
              labels={"x":"statut du suspect", "y":"nombre de suspects (échelle logarithmique)"},
              title="Nombre de suspect par statut",
              log_y=True)

fig7

In [28]:
fig9 = px.bar(filtered_suspects_status, x=sorted(pd.unique(filtered_suspects_status['status_sus'])), y=filtered_suspects_status.groupby('status_sus').count()['id'],
              labels={"x":"statut du suspect", "y":"nombre de suspects"},
              title="Nombre de suspect par statut")

fig9

### Statistiques sur les caractéristiques des incidents 

In [138]:
liste_carac=[]
for i in updated_base["incident_characteristics"]:
    if type(i) != list:
        pass
    else:
        liste_carac.append(i)
flat_carac=[item for sublist in liste_carac for item in sublist]
pd.unique(flat_carac)

array(['Shot - Wounded/Injured',
       'Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)',
       'Possession (gun(s) found during commission of other crimes)',
       'Possession of gun by felon or prohibited person',
       'Shot - Dead (murder, accidental, suicide)', 'Gang involvement',
       'Shots Fired - No Injuries',
       'Bar/club incident - in or around establishment',
       'Officer Involved Incident',
       'Officer Involved Shooting - subject/suspect/perpetrator killed',
       'Drug involvement', 'Kidnapping/abductions/hostage',
       'Under the influence of alcohol or drugs (only applies to the subject/suspect/perpetrator )',
       'Suicide^', 'Murder/Suicide',
       'Attempted Murder/Suicide (one variable unsuccessful)',
       'Domestic Violence', 'Home Invasion',
       'Home Invasion - Resident killed',
       'Armed robbery with injury/death and/or evidence of DGU found',
       'Drive-by (car to street, ca

On voit ici que ces données liées aux caractéristiques sont des champs libres non-normalisés donc on va d'abord s'intéresser aux descriptions comportant la mention de suicides car c'est, selon le Centers for Disease Control and Prevention's (CDC) National Center for Health Statistics, la cause principale de mort lors des incidents par arme à feu.

In [69]:
#création d'une indicatrice signalant la description d'un suicide

liste_indic=[]

for i in updated_base['incident_characteristics']:
    if i is not None:
        if "suicide" in str(i) :
            liste_indic.append(1)
        else:
            liste_indic.append(0)
    else:
        liste_indic.append(0)
    

updated_base['indic_suicide']=liste_indic
updated_base.head()

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0
1,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},NaN,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0,1-2013,1
2,2,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,NaN,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",NaN,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0,1-2013,1
3,3,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,NaN,NaN,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,NaN,-104.8020,NaN,NaN,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",NaN,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0,1-2013,1
4,4,478959,2013-01-07,North Carolina,Greensboro,307 Mourning Dove Terrace,2,2,http://www.gunviolencearchive.org/incident/478959,http://www.journalnow.com/news/local/article_d...,False,6.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Handgun', 1: 'Handgun'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",36.1140,NaN,-79.9569,2.0,Two firearms recovered. (Attempted) murder sui...,"{0: '18', 1: '46', 2: '14', 3: '47'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Teen 12-1...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Female'}","{0: 'Danielle Imani Jameison', 1: 'Maurice Eug...",{0: '3::Family'},"{0: 'Injured', 1: 'Injured', 2: 'Killed', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://myfox8.com/2013/01/08/update-mother-sh...,62.0,27.0,1-2013,1


In [31]:
fig10 = px.bar(updated_base[updated_base['n_killed']>=1], x=pd.unique(updated_base['indic_suicide']), y=updated_base[updated_base['n_killed']>=1].groupby(updated_base['indic_suicide']).count()['incident_id'],
              labels={"x":"incident impliquant un suicide (1) ou non (0)", "y":"nombre d'incidents"},
              title="Parmi les incidents impliquant au moins un mort, combien sont le résultat d'un suicide ?")

fig10

### Statistiques descriptives sur les victimes des incidents 

On commence par recréer une base similaire à celle des suspects pour récupérer les informations présentes dans les dictionnaires.

In [70]:
data_victimes=[]
index=0

for i in updated_base['participant_type']:
    if type(i)==dict and len(i)>0:
        for key,value in i.items():
            if value=='Victim':
                result_dict={'id':index}

                if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
                else:
                    result_dict['age_group_vict']=np.nan
                
                if type(updated_base['participant_age'][index])==dict:
                    if key in updated_base['participant_age'][index].keys():
                        result_dict['age_vict']=updated_base['participant_age'][index][key]
                else:
                    result_dict['age_vict']=np.nan

                if type(updated_base['participant_gender'][index])==dict:
                    if key in updated_base['participant_gender'][index].keys():
                        result_dict['gender_vict']=updated_base['participant_gender'][index][key]
                else:
                    result_dict['gender_vict']=np.nan

                if type(updated_base['participant_status'][index])==dict:
                    if key in updated_base['participant_status'][index].keys():
                        result_dict['status_vict']=updated_base['participant_status'][index][key]
                else:
                    result_dict['status_vict']=np.nan

                if type(updated_base['gun_type'][index])==dict:
                    if key in updated_base['gun_type'][index].keys():
                        result_dict['gun_type']=updated_base['gun_type'][index][key]
                else:
                    result_dict['gun_type']=np.nan

                if type(updated_base['gun_stolen'][index])==dict:
                    if key in updated_base['gun_stolen'][index].keys():
                        result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
                else:
                    result_dict['gun_stolen']=np.nan
            
                
                data_victimes.append(result_dict)


    elif np.isnan(i)==False:
        if updated_base['participant_type'][i]=='Victim':
            result_dict={'id':index}
            if type(updated_base['participant_age_group'][index])==dict:
                    if key in updated_base['participant_age_group'][index].keys():
                        result_dict['age_group_vict']=updated_base['participant_age_group'][index][key]
            else:
                result_dict['age_group_vict']=np.nan
            
            if type(updated_base['participant_age'][index])==dict:
                if key in updated_base['participant_age'][index].keys():
                    result_dict['age_vict']=updated_base['participant_age'][index][key]
            else:
                result_dict['age_vict']=np.nan

            if type(updated_base['participant_gender'][index])==dict:
                if key in updated_base['participant_gender'][index].keys():
                    result_dict['gender_vict']=updated_base['participant_gender'][index][key]
            else:
                result_dict['gender_vict']=np.nan

            if type(updated_base['participant_status'][index])==dict:
                if key in updated_base['participant_status'][index].keys():
                    result_dict['status_vict']=updated_base['participant_status'][index][key]
            else:
                result_dict['status_vict']=np.nan

            if type(updated_base['gun_type'][index])==dict:
                if key in updated_base['gun_type'][index].keys():
                    result_dict['gun_type']=updated_base['gun_type'][index][key]
            else:
                result_dict['gun_type']=np.nan

            if type(updated_base['gun_stolen'][index])==dict:
                if key in updated_base['gun_stolen'][index].keys():
                    result_dict['gun_stolen']=updated_base['gun_stolen'][index][key]
            else:
                result_dict['gun_stolen']=np.nan


            data_victimes.append(result_dict)

    index+=1


df_victim=pd.DataFrame(data_victimes)

In [71]:
df_victim.head()

,id,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen
0,0,Adult 18+,0::20,Male,Arrested,NaN,NaN
1,0,Adult 18+,NaN,Male,Injured,NaN,NaN
2,0,Adult 18+,NaN,NaN,Injured,NaN,NaN
3,0,Adult 18+,NaN,Male,Injured,NaN,NaN
4,1,Adult 18+,0::20,0::Male,Killed,NaN,NaN


On va faire des graphiques de statistiques descriptives similaires à ceux concernant les suspects:

In [34]:
groupes=['Adult 18+', 'Teen 12-17','Child 0-11']
filtered_victim_age=df_victim[df_victim['age_group_vict'].isin(groupes)]

fig11 = px.bar(filtered_victim_age, x=sorted(pd.unique(filtered_victim_age['age_group_vict'])), y=filtered_victim_age.groupby('age_group_vict').count()['id'],
              labels={"x":"âge de la victime", "y":"nombre de victimes"},
              title="Nombre de victimes par tranche d'âge")

fig11

In [72]:
filtered_victim_genre=df_victim[df_victim['gender_vict'].isin(gender)]

fig12 = px.bar(filtered_victim_genre, x=sorted(pd.unique(filtered_victim_genre['gender_vict'])), y=filtered_victim_genre.groupby('gender_vict').count()['id'],
              labels={"x":"genre de la victime", "y":"nombre de victimes"},
              title="Genre des victimes")

fig12

In [36]:
stat_vict=['Arrested', 'Injured', 'Killed', 'Unharmed',
       'Injured, Unharmed', 'Killed, Injured', 'Injured, Arrested', 'Unharmed, Arrested', 'Killed, Arrested',
       'Killed, Unharmed','Killed, Unharmed, Arrested', 'Injured, Unharmed, Arrested']
filtered_vict_status=df_victim[df_victim['status_vict'].isin(statut)]

fig13 = px.bar(filtered_vict_status, x=sorted(pd.unique(filtered_vict_status['status_vict'])), y=filtered_vict_status.groupby('status_vict').count()['id'],
              labels={"x":"statut de la victime", "y":"nombre de victimes"},
              title="Statut des victimes")

fig13

In [37]:
fig14 = px.bar(filtered_vict_status, x=sorted(pd.unique(filtered_vict_status['status_vict'])), y=filtered_vict_status.groupby('status_vict').count()['id'],
              labels={"x":"statut de la victime", "y":"nombre de victimes (échelle logarithmique)"},
              title="Statut des victimes", log_y=True)

fig14

In [80]:
age=list(range(100))
age=[str(num) for num in age]

filtered_vict_age2 = df_victim[df_victim['age_vict'].isin(age)].copy()  

filtered_vict_age2.loc[:, 'age_vict'] = filtered_vict_age2['age_vict'].astype(int)

fig15 = px.bar(filtered_vict_age2, x=sorted(pd.unique(filtered_vict_age2['age_vict'])), y=filtered_vict_age2.groupby('age_vict').count()['id'],
            labels={"x":"âge de la victime", "y":"nombre de victimes"},
            title="Nombre de victimes par âge")

fig15

### Statistiques descriptives concernant les armes à feu utilisées.

In [39]:

list_guns=['Unknown', 'Handgun', '22 LR', '223 Rem [AR-15]',
    '9mm', '40 SW', 'Shotgun', '7.62 [AK-47]','45 Auto', '44 Mag','410 gauge', '32 Auto',
    'Other','Rifle','357 Mag', '38 Spl','25 Auto','12 gauge', '30-30 Win',
    '380 Auto','308 Win','20 gauge','16 gauge','30-06 Spr','300 Win', '10mm','28 gauge']

filtered_suspects_guns=df_suspects[df_suspects['gun_type'].isin(list_guns)]

fig16 = px.bar(filtered_suspects_guns, x=sorted(pd.unique(filtered_suspects_guns['gun_type'])), y=filtered_suspects_guns.groupby('gun_type').count()['id'],
              labels={"x":"arme utilisée", "y":"nombre de suspects concernés"},
              title="Nombre de suspect concernés par un incident avec ces armes")

fig16


In [40]:
fig17 = px.bar(filtered_suspects_guns, x=sorted(pd.unique(filtered_suspects_guns['gun_type'])), y=filtered_suspects_guns.groupby('gun_type').count()['id'],
              labels={"x":"arme utilisée", "y":"nombre de suspects concernés"},
              title="Nombre de suspect concernés par un incident avec ces armes",
              log_y=True)

fig17

In [88]:
df_suspects[df_suspects['gun_stolen'].isin(['Stolen', 'Not-Stolen'])].shape

(1991, 8)

Le nombre de suspects pour lesquels l'information sur le vol de l'arme à feu est disponible est très faible (1991), cette information semble donc difficile à exploiter.

### Mise en commun des bases sur les participants dans une seule base

In [81]:
updated_base=updated_base.reset_index()
merge_sus=pd.merge(updated_base, df_suspects, how='outer', left_on='index', right_on='id')
merge_tot=pd.merge(merge_sus, df_victim, how='outer', right_on='id', left_on='index')
merge_tot.head()

,level_0,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen
0,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,0::20,Male,Arrested,NaN,NaN
1,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN
2,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN
3,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN
4,1,1,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,NaN,NaN,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,NaN,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gilli

In [42]:
pd.unique(merge_tot['gun_type'])

array([nan, 'Handgun', '22 LR', '223 Rem [AR-15]', '0::Unknown',
       '0::Shotgun', '0::Handgun', '0::9mm', '0::45 Auto', '0::12 gauge',
       '0::7.62 [AK-47]', '40 SW', 'Shotgun', '7.62 [AK-47]', '9mm',
       'Unknown', '45 Auto', '0::44 Mag', '44 Mag', '0::Other',
       '0::40 SW', '0::38 Spl', '0::22 LR', '0::380 Auto', '410 gauge',
       '32 Auto', '0::308 Win', '0::357 Mag', 'Rifle', '357 Mag',
       '38 Spl', '0::30-30 Win', '30-30 Win', '0::Rifle',
       '0::223 Rem [AR-15]', '0:Unknown', '0:22 LR', '0:25 Auto', '0:9mm',
       '0:45 Auto', '0:Shotgun', '0:20 gauge', '0:357 Mag', '0:12 gauge',
       '0:40 SW', '0:Handgun', '0:44 Mag', '0:38 Spl', '0::32 Auto',
       '0::10mm', '0::20 gauge', 'Other', '0::25 Auto', '25 Auto',
       '12 gauge', '380 Auto', '308 Win', '0::28 gauge', '20 gauge',
       '0::300 Win', '0::410 gauge', '0::30-06 Spr', '0::16 gauge',
       '30-06 Spr', '10mm'], dtype=object)

In [43]:
list_guns=['Unknown', 'Handgun', '22 LR', '223 Rem [AR-15]',
    '9mm', '40 SW', 'Shotgun', '7.62 [AK-47]','45 Auto', '44 Mag','410 gauge', '32 Auto',
    'Other','Rifle','357 Mag', '38 Spl','25 Auto','12 gauge', '30-30 Win',
    '380 Auto','308 Win','20 gauge','16 gauge','30-06 Spr','300 Win', '10mm','28 gauge']

filtered_guns=merge_tot[merge_tot['gun_type'].isin(list_guns)]

fig18 = px.bar(merge_tot, x=sorted(pd.unique(filtered_guns['gun_type'])), y=filtered_guns[['gun_type', 'n_injured']].groupby('gun_type').mean()['n_injured'],
              labels={"x":"arme utilisée", "y":"nombre de blessés moyen"},
              title="Moyenne du nombre de blessés en fonction de l'arme utilisée")

fig18


A prendre avec grosses pincettes : database de 8000 incidents NON-UNIQUES (et dont le problème de non-unicité n'est ici pas géré donc certains évènements sont comptabilisés autant de fois qu'il y a de suspects).

### Statistiques descriptives concernant les relations de l'auteur des faits avec les victimes

La base de données filtrée ne comprenant que les suspects pour lesquels leur relation aux victimes a été bien renseignée comporte 7920 lignes, ce qui est très peu relativement à la base de données initiale mais reste assez important pour mettre en évidence le fait qu'une majorité des incidents pour lesquels cette donnée est renseignée sont des "Armed Robbery".

In [55]:
relations=['Family','Drive by - Random victims','Gang vs Gang','Armed Robbery','Home Invasion - Perp Does Not Know Victim',
    'Aquaintance', 'Significant others - current or former' ,'Home Invasion - Perp Knows Victim','Friends' ,
    'Mass shooting - Perp Knows Victims', 'Mass shooting - Random victims','Neighbor',
    'Co-worker']

filtered_suspects_relations=df_suspects[df_suspects['relation'].isin(relations)]

fig19 = px.bar(filtered_suspects_relations, x=sorted(pd.unique(filtered_suspects_relations['relation'])), y=filtered_suspects_relations.groupby('relation').count()['id'],
              labels={"x":"type de relation", "y":"nombre de relations concernés"},
              title="Nombre d'incidents dans lesquels ces relations/situations ont joué un rôle")

fig19

In [56]:
fig20 = px.bar(filtered_suspects_relations, x=sorted(pd.unique(filtered_suspects_relations['relation'])), y=filtered_suspects_relations.groupby('relation').count()['id'],
              labels={"x":"type de relation", "y":"nombre de relations concernées"},
              title="Nombre d'incidents dans lesquels ces relations/situations ont joué un rôle",
              log_y=True)

fig20

### Export de la base en csv pour pouvoir l'utiliser si besoin

In [60]:
merge_tot.to_csv("df_participants.csv")

### Création d'indicatrices sur les variables d'intérêt identifiées précédemment 

Avoir des indicatrices sur ces sujets permettra d'obtenir plus facilement certaines informations clefs qui seront importantes lorsque l'on va croiser cette base de données avec des données extérieures

In [97]:
def make_indic(var, val1, new_name):
    list_indic=[]
    for i in data_indic[var]:
        
        if i!=None:
            if type(i)==str:
                if val1 in i :
                    list_indic.append(1)
                else:
                    list_indic.append(0)
            else:
                list_indic.append(np.nan)
        else:
            list_indic.append(np.nan)
    data_indic[new_name]=list_indic

In [91]:
data_indic=merge_tot.copy()

In [100]:
to_indic=[('age_group_sus', 'Adult 18+', 'sus_adulte'),
    ('age_group_vict', 'Adult 18+', 'vict_adulte'),
    ('gender_sus', 'Male', 'sus_male'),
    ('gender_vict', 'Male', 'vict_male'),
    ('status_sus', 'Injured', 'sus_injured'),
    ('status_sus', 'Arrested', 'sus_arrested'),
    ('status_sus', 'Unharmed', 'sus_unharmed'),
    ('status_sus', 'Killed', 'sus_killed'),
    ('status_vict', 'Injured', 'vict_injured'),
    ('status_vict', 'Arrested', 'vict_arrested'),
    ('status_vict', 'Unharmed', 'vict_unharmed'),
    ('status_vict', 'Killed', 'vict_killed'),
    ('gun_type', 'Handgun', 'handgun_1'),
    ('gun_type', 'Rifle', 'rifle_1'),
    ('gun_type', '9mm', '9mm_1'),
    ('gun_type', 'Shotgun', 'shotgun_1'),
    ('gun_type_y', 'Handgun', 'handgun_2'),
    ('gun_type_y', 'Rifle', 'rifle_2'),
    ('gun_type_y', '9mm', '9mm_2'),
    ('gun_type_y', 'Shotgun', 'shotgun_2')]

for col, val1, new_name in to_indic: 
    make_indic(col, val1, new_name)

In [101]:
data_indic.head()

,level_0,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun_1,rifle_1,9mm_1,shotgun_1,handgun_2,rifle_2,9mm_2,shotgun_2
0,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,0::20,Male,Arrested,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+

In [105]:
#taking care of the issue of the gun_type variable

def one_col(col1, col2, new_col):
    data_indic[new_col] = np.where(
        (data_indic[col1] == 1) | (data_indic[col2] == 1),  # If either column is 1
        1,
        np.where(
            (data_indic[col1] == 0) | (data_indic[col2] == 0),  # If either column is 0
            0,
            np.nan  # If both columns are NaN
        )
    )

In [107]:
gun_type_list=[('handgun_1', 'handgun_2', 'handgun'),
    ('rifle_1','rifle_2','rifle'),
    ('9mm_1', '9mm_2', '9mm'),
    ('shotgun_1','shotgun_2','shotgun')]

for col1, col2, new_col in gun_type_list:
    one_col(col1,col2, new_col)

In [110]:
data_indic.head()

,level_0,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun_1,rifle_1,9mm_1,shotgun_1,handgun_2,rifle_2,9mm_2,shotgun_2,handgun,rifle,9mm,shotgun
0,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,0::20,Male,Arrested,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,N

In [112]:
del data_indic['9mm_1'], data_indic['9mm_2'], data_indic['rifle_1'], data_indic['rifle_2'], data_indic['handgun_1'], data_indic['handgun_2'], data_indic['shotgun_1'], data_indic['shotgun_2']

In [113]:
data_indic.head()

,level_0,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,9mm,shotgun
0,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,0::20,Male,Arrested,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN
1,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
3,0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN


In [119]:
young=list(range(18,26))
young=[str(num) for num in young]

def indic_age(var, list1, new_name):
    list_indic=[]
    for i in data_indic[var]:
        
        if i!=None:
            if type(i)==str:
                if i in list1 :
                    list_indic.append(1)
                else:
                    list_indic.append(0)
            else:
                list_indic.append(np.nan)
        else:
            list_indic.append(np.nan)
    data_indic[new_name]=list_indic

indic_age('age_sus', young, 'young_sus')
indic_age('age_vict', young, 'young_vict')

In [125]:
del data_indic['level_0']

In [124]:
col=list(data_indic.columns)
col

['level_0',
 'index',
 'incident_id',
 'date',
 'state',
 'city_or_county',
 'address',
 'n_killed',
 'n_injured',
 'incident_url',
 'source_url',
 'incident_url_fields_missing',
 'congressional_district',
 'gun_stolen_x',
 'gun_type_x',
 'incident_characteristics',
 'latitude',
 'location_description',
 'longitude',
 'n_guns_involved',
 'notes',
 'participant_age',
 'participant_age_group',
 'participant_gender',
 'participant_name',
 'participant_relationship',
 'participant_status',
 'participant_type',
 'sources',
 'state_house_district',
 'state_senate_district',
 'month_year',
 'indic_suicide',
 'id_x',
 'age_group_sus',
 'gender_sus',
 'status_sus',
 'gun_type_y',
 'gun_stolen_y',
 'relation',
 'age_sus',
 'id_y',
 'age_group_vict',
 'age_vict',
 'gender_vict',
 'status_vict',
 'gun_type',
 'gun_stolen',
 'sus_adulte',
 'vict_adulte',
 'sus_male',
 'vict_male',
 'sus_injured',
 'sus_arrested',
 'sus_unharmed',
 'sus_killed',
 'vict_injured',
 'vict_arrested',
 'vict_unharmed',
 

In [126]:
data_indic.head()

,index,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen_x,gun_type_x,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,id_x,age_group_sus,gender_sus,status_sus,gun_type_y,gun_stolen_y,relation,age_sus,id_y,age_group_vict,age_vict,gender_vict,status_vict,gun_type,gun_stolen,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,9mm,shotgun,young_sus,young_vict
0,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,0::20,Male,Arrested,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0
1,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,NaN,Injured,NaN,NaN,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,0,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,NaN,NaN,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,NaN,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},NaN,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,0.0,Adult 18+,Female,Injured,NaN,NaN,NaN,NaN,0.0,Adult 18+,NaN,Male,Injured,NaN,NaN,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0

Maintenant que toutes les variables d'intérêt sont présentes dans le dataframe, on revient au format initial pour lequel il y avait une seule ligne par incident.

In [129]:
aggregated_data = data_indic.groupby('incident_id').agg(
    date=('date','first'), #using first to get the first non-null value
    state=('state','first'),
    city_or_county=('city_or_county','first'),
    address=('address','first'),
    n_killed=('n_killed', 'first'),
    n_injured=('n_injured','first'),
    incident_url=('incident_url','first'),
    source_url=('source_url','first'),
    incident_url_fields_missing=('incident_url_fields_missing','first'),
    congressional_district=('congressional_district','first'),
    gun_stolen=('gun_stolen_x','first'),
    gun_type=('gun_type_x','first'),
    incident_characteristics=('incident_characteristics','first'),
    latitude=('latitude','first'),
    location_description=('location_description','first'),
    longitude=('longitude','first'),
    n_guns_involved=('n_guns_involved','first'),
    notes=('notes','first'),
    participant_age=('participant_age','first'),
    participant_age_group=('participant_age_group','first'),
    participant_gender=('participant_gender','first'),
    participant_name=('participant_name','first'),
    participant_relationship=('participant_relationship','first'),
    participant_status=('participant_status','first'),
    participant_type=('participant_type','first'),
    sources=('sources','first'),
    state_house_district=('state_house_district','first'),
    state_senate_district=('state_senate_district','first'),
    month_year=('month_year','first'),
    indic_suicide=('indic_suicide','first'),
    sus_adulte=('sus_adulte', lambda x: 1 if x.sum() >= 1 else 0), #custom lambda function used to indicate the presence instead of counting the indicators (because it would give corrupted answers because of the merged data)
    vict_adulte=('vict_adulte', lambda x: 1 if x.sum() >= 1 else 0),
    sus_male=('sus_male', lambda x: 1 if x.sum() >= 1 else 0),
    vict_male=('vict_male', lambda x: 1 if x.sum() >= 1 else 0),
    sus_injured=('sus_injured', lambda x: 1 if x.sum() >= 1 else 0),
    sus_arrested=('sus_arrested', lambda x: 1 if x.sum() >= 1 else 0),
    sus_unharmed=('sus_unharmed', lambda x: 1 if x.sum() >= 1 else 0),
    sus_killed=('sus_killed', lambda x: 1 if x.sum() >= 1 else 0),
    vict_injured=('vict_injured', lambda x: 1 if x.sum() >= 1 else 0),
    vict_arrested=('vict_arrested', lambda x: 1 if x.sum() >= 1 else 0),
    vict_unharmed=('vict_unharmed', lambda x: 1 if x.sum() >= 1 else 0),
    vict_killed=('vict_killed', lambda x: 1 if x.sum() >= 1 else 0),
    handgun=('handgun', lambda x: 1 if x.sum() >= 1 else 0),
    rifle=('rifle', lambda x: 1 if x.sum() >= 1 else 0),
    Nine_mm=('9mm', lambda x: 1 if x.sum() >= 1 else 0),
    shotgun=('shotgun', lambda x: 1 if x.sum() >= 1 else 0),
    young_sus=('young_sus', lambda x: 1 if x.sum() >= 1 else 0),
    young_vict=('young_vict', lambda x: 1 if x.sum() >= 1 else 0)
)

aggregated_data.reset_index(inplace=True)

In [130]:
aggregated_data

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
0,92114,2014-01-01,Kentucky,Lexington,Sixth Street and Elm Tree Lane,0,1,http://www.gunviolencearchive.org/incident/92114,http://www.lex18.com/news/lexington-police-sta...,False,6.0,None,None,None,38.0507,None,-84.4829,NaN,None,None,{0: '0::Adult 18+'},{0: '0::Male'},None,None,{0: '0::Injured'},{0: '0::Victim'},None,NaN,NaN,1-2014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,92117,2014-01-01,Kentucky,Cynthiana,None,0,1,http://www.gunviolencearchive.org/incident/92117,https://www.wtvq.com/2014/01/01/harrison-sheri...,False,4.0,{0: '0::Unknown'},{0: '0::Unknown'},"[Shot - Wounded/Injured, Accidental Shooting, ...",38.4333,None,-84.3542,1.0,US-62 east of Cynthiana,{0: '1::22'},"{0: 'Adult 18+', 1: 'Adult 18+'}","{0: 'Male', 1: 'Male'}","{0: 'Dustin Mullins', 1: 'Timothy Turner'}",{0: '1::Friends'},"{0: 'Injured', 1: 'Unharmed, Arrested'}","{0: 'Victim', 1: 'Subject-Suspect'}",[http://www.lex18.com/news/man-arrested-in-con...,78.0,27.0,1-2014,0,1,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0
2,92119,2014-01-01,Kentucky,Louisville,38th and Broadway,0,1,http://www.gunviolencearchive.org/incident/92119,http://www.whas11.com/news/local/Person-injure...,False,3.0,None,None,None,38.2501,None,-85.8150,NaN,None,None,{0: '0::Adult 18+'},{0: '0::Male'},None,None,{0: '0::Injured'},{0: '0::Victim'},None,NaN,NaN,1-2014,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,92122,2014-01-01,Virginia,Falls Church,North Washington Street,0,0,http://www.gunviolencearchive.org/incident/92122,http://www.nbcwashington.com/news/local/Two-So...,False,8.0,None,None,None,38.8847,None,-77.1672,NaN,None,None,None,None,None,None,{0: '0::Unharmed'},"{0: 'Victim', 1: 'Subject-Suspect'}",None,53.0,35.0,1-2014,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,92125,2014-01-01,Oklahoma,Lawton,Sheridan and Pollard,0,0,http://www.gunviolencearchive.org/incident/92125,http://www.kswo.com/story/24342150/lpd-investi...,False,4.0,None,None,None,34.6347,None,-98.4222,NaN,None,None,{0: '0::Adult 18+'},"{0: 'Male', 1: 'Male'}",None,None,"{0: 'Unharmed', 1: 'Unharmed'}","{0: 'Victim', 1: 'Subject-Suspect'}",None,64.0,32.0,1-2014,0,0,1,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239672,1083428,2018-03-30,Illinois,Centralia,E Noleman St and N Maple St,0,1,http://www.gunviolencearchive.org/incident/108...,https://mvnnews.wordpress.com/2018/04/02/centr...,False,15.0,{0: '0::Unknown'},{0: '0::Unknown'},None,38.5281,None,-89.1295,1.0,Known suspect sought;,None,{0: '0::Adult 18+'},"{0: 'Male', 1: 'Male'}",None,None,"{0: 'Injured', 1: 'Unharmed'}","{0: 'Victim', 1: 'Subject-Suspect'}",None,107.0,54.0,3-2018,0,0,1,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0
239673,1083435,2018-03-20,West Virginia,Williamson,Vinson St,0,0,http://www.gunviolencearchive.org/incident/108...,http://www.williamsondailynews.com/news/police...,False,3.0,{0: '0::Unknown'},{0: '0::Unknown'},"[Shots Fired - No Injuries, Brandishing/flouri...",37.6750,None,-82.2672,1.0,Multiple shots fired through door,{0: '0::18'},{0: '0::Adult 18+'},{0: '0::Male'},{0: '0::Armando Israel Elizondo'},None,{0: '0::Arrested'},{0: '0::Subject-Suspect'},None,20.0,6.0,3-2018,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
239674,1083457,2018-03-22,District of Columbia,

In [134]:
#sorting the dataframe by date to get the same order as in the original dataframe
agg_sorted = aggregated_data.sort_values(by='date', ascending=True)
agg_sorted

,incident_id,date,state,city_or_county,address,n_killed,n_injured,incident_url,source_url,incident_url_fields_missing,congressional_district,gun_stolen,gun_type,incident_characteristics,latitude,location_description,longitude,n_guns_involved,notes,participant_age,participant_age_group,participant_gender,participant_name,participant_relationship,participant_status,participant_type,sources,state_house_district,state_senate_district,month_year,indic_suicide,sus_adulte,vict_adulte,sus_male,vict_male,sus_injured,sus_arrested,sus_unharmed,sus_killed,vict_injured,vict_arrested,vict_unharmed,vict_killed,handgun,rifle,Nine_mm,shotgun,young_sus,young_vict
100178,461105,2013-01-01,Pennsylvania,Mckeesport,1506 Versailles Avenue and Coursin Street,0,4,http://www.gunviolencearchive.org/incident/461105,http://www.post-gazette.com/local/south/2013/0...,False,14.0,None,None,"[Shot - Wounded/Injured, Mass Shooting (4+ vic...",40.3467,None,-79.8559,NaN,Julian Sims under investigation: Four Shot and...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 3: 'Male', 4: 'Female'}",{0: '0::Julian Sims'},None,"{0: 'Arrested', 1: 'Injured', 2: 'Injured', 3:...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://pittsburgh.cbslocal.com/2013/01/01/4-p...,NaN,NaN,1-2013,0,1,1,0,1,1,0,0,0,1,1,0,0,0,0,0,0,0,0
104318,478855,2013-01-01,Ohio,Lorain,1776 East 28th Street,1,3,http://www.gunviolencearchive.org/incident/478855,http://chronicle.northcoastnow.com/2013/02/14/...,False,9.0,"{0: 'Unknown', 1: 'Unknown'}","{0: 'Unknown', 1: 'Unknown'}","[Shot - Wounded/Injured, Shot - Dead (murder, ...",41.4455,Cotton Club,-82.1377,2.0,None,"{0: '25', 1: '31', 2: '33', 3: '34', 4: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Male', 1: 'Male', 2: 'Male', 3: 'Male', 4...","{0: 'Damien Bell', 1: 'Desmen Noble', 2: 'Herm...",None,"{0: 'Injured, Unharmed, Arrested', 1: 'Unharme...","{0: 'Subject-Suspect', 1: 'Subject-Suspect', 2...",[http://www.morningjournal.com/general-news/20...,56.0,13.0,1-2013,1,1,1,1,1,1,1,1,0,1,0,0,1,0,0,0,0,1,0
100093,460726,2013-01-01,California,Hawthorne,13500 block of Cerise Avenue,1,3,http://www.gunviolencearchive.org/incident/460726,http://www.dailybulletin.com/article/zz/201301...,False,43.0,None,None,"[Shot - Wounded/Injured, Shot - Dead (murder, ...",33.9090,None,-118.3330,NaN,Four Shot; One Killed; Unidentified shooter in...,{0: '0::20'},"{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...",{0: '0::Male'},{0: '0::Bernard Gillis'},None,"{0: 'Killed', 1: 'Injured', 2: 'Injured', 3: '...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Vi...",[http://losangeles.cbslocal.com/2013/01/01/man...,62.0,35.0,1-2013,1,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0
104325,478925,2013-01-05,Colorado,Aurora,16000 block of East Ithaca Place,4,0,http://www.gunviolencearchive.org/incident/478925,http://www.dailydemocrat.com/20130106/aurora-s...,False,6.0,None,None,"[Shot - Dead (murder, accidental, suicide), Of...",39.6518,None,-104.8020,NaN,None,"{0: '29', 1: '33', 2: '56', 3: '33'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Male', 2: 'Male', 3: 'Male'}","{0: 'Stacie Philbrook', 1: 'Christopher Ratlif...",None,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victim', 3: 'Su...",[http://denver.cbslocal.com/2013/01/06/officer...,40.0,28.0,1-2013,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0
104332,478948,2013-01-07,Oklahoma,Tulsa,6000 block of South Owasso,4,0,http://www.gunviolencearchive.org/incident/478948,http://usnews.nbcnews.com/_news/2013/01/07/163...,False,1.0,None,None,"[Shot - Dead (murder, accidental, suicide), Ho...",36.2405,Fairmont Terrace,-95.9768,NaN,None,"{0: '23', 1: '23', 2: '33', 3: '55'}","{0: 'Adult 18+', 1: 'Adult 18+', 2: 'Adult 18+...","{0: 'Female', 1: 'Female', 2: 'Female', 3: 'Fe...","{0: 'Rebeika Powell', 1: 'Kayetie Melchor', 2:...",None,"{0: 'Killed', 1: 'Killed', 2: 'Killed', 3: 'Ki...","{0: 'Victim', 1: 'Victim', 2: 'Victi

In [135]:
agg_sorted.to_csv('df_gun_violence.csv')